# Introduction:

In this project, we have tried to implement a **CHATBOT** using **Natural Language Processing (NLP)** and **Neural Networks**. 

A **chatbot** is a software application used to conduct an online chat conversation via text or text-to-speech, in lieu of providing direct contact with a live human agent. It is designed to convincingly simulate the way a human would behave as a conversational partner. 

We have created a chatbot for Data Structures and Algorithms. A **User** can ask anything related to Data Structures and Algorithms and the **Bot** will answer it accordingly. 

# Importing Libraries:

We have used various Python Modules in this Notebook.
* `nltk` refers to Natural Language Toolkit and helps to perform NLP
* `tensorflow` is a free and open-source software library for machine learning. It's primary focus is on training and inference of deep neural networks.
* `tflearn` is a modular and transparent deep learning library built on top of Tensorflow.

In [1]:
import json
import nltk
import numpy
import random
import tensorflow
import tflearn
import pickle
from termcolor import colored
from nltk.stem.lancaster import LancasterStemmer
nltk.download('punkt')
stemmer = LancasterStemmer()

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


[nltk_data] Downloading package punkt to C:\Users\KUMAR
[nltk_data]     APURV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Dataset:

We have created **`intents.json`** which is a json file containing our training data that trains our model on Data Structures and Algorithms. It contains a dictionary with **key** as `intents` and **value** as a `list of dictionaries`. In the list, each dictionary have three keys which are:
* `tag`: It represents the topic.
* `patterns`: It contains various ways of asking the questions related to that topic.
* `responses`: It contains the answers to that questions.

Here, we have loaded the intents.json file.

In [2]:
with open('intents.json') as file:
    data = json.load(file)

* `'words'` is a bag of words which contains all the stemmed words. For this purpose, we have used Tokenization and Stemming. 

* `'labels'` are used for storing all the tags. 

* `'training'` is a 2D matrix in which each row is a binary representation of a pattern. Each row contains value as **1** where the words of pattern matches with the words in 'bag of words' otherwise **0**.

* `'output'` is a 2D matrix used to map each row of training to it's corresponding tag.

In [3]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)
    

Finally, we have created a **Deep Neural Network**. It contains an `input layer`, `5 hidden layers` each containing 100 features and an `output layer`. It is a fully connected network. 

In [4]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Here, we are searching if we already have a trained model. If yes, we will load the model, otherwise we will train our model.

In [5]:
try:
    model.load('model.tflearn')
except:
    from tensorflow.python.framework import ops
    ops.reset_default_graph()

    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 100)
    net = tflearn.fully_connected(net, 100)
    net = tflearn.fully_connected(net, 100)
    net = tflearn.fully_connected(net, 100)
    net = tflearn.fully_connected(net, 100)
    net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
    net = tflearn.regression(net)

    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


INFO:tensorflow:Restoring parameters from C:\Users\KUMAR APURV\Desktop\Chatbot\model.tflearn


This function is used to convert the input string (pattern) into it's binary representation.

In [6]:
def convert(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

This the function used to call the chatbot.

In [7]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        print()
        print(colored("You:",'blue'))
        inp = input()
        if inp.lower() == "quit":
            break

        results = model.predict([convert(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if (results[results_index] > 0.8):
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(colored("Bot: ",'red'))
            final_response = random.choice(responses)
            print(final_response)
            
        else:
            print(colored("Bot: ",'red'))
            print("I'm not sure about that. Try again.")


In [8]:
chat()

Start talking with the bot (type quit to stop)!

You:
Hi
Bot: 
Hello

You:
Who are you?
Bot: 
I am a chatbot

You:
What is Data Structures?
Bot: 
A data structure is a particular way of organizing data in a computer so that it can be used effectively.

You:
Tell me about Linked List
Bot: 
A linked list is a linear data structure, in which the elements are not stored at contiguous memory locations. 
The elements in a linked list are linked using pointers. 
There are 3 different implementations of Linked List available, they are
1. Singly Linked List
2. Doubly Linked List
3. Circular Linked List

You:
Do you know about queues?
Bot: 
A Queue is a linear structure which follows a particular order in which the operations are performed. 
The order is First In First Out (FIFO). 
A good example of a queue is any queue of consumers for a resource where the consumer that came first is served first. 
The difference between stacks and queues is in removing. In a stack we remove the item the most r